In [1]:
from tools import get_input
import numpy as np

data, data_test = get_input(20)
len(data), len(data_test)

(58, 5)

In [38]:
f = open(f"data/20_test.txt", "r")
data_test = f.readlines()
data_test = [l[:-1] for l in data_test]

f = open(f"data/20_test2.txt", "r")
data_test2 = f.readlines()
data_test2 = [l[:-1] for l in data_test2]

In [111]:
L = data_test

def initialize(L):
    states = {}
    conj = set()
    destinations = {}
    for line in L:
        name, sdest = [s.strip() for s in line.split("->")]
        dest = [s.strip() for s in sdest.split(",")]
        if name != "broadcaster":
            prefix = name[0]
            id = name[1:]
            if prefix == "%":
                states[id] = 0
            elif prefix == "&":
                conj.add(id)
        else:
            id = name
        destinations[id] = dest

    memory = {}
    for name in conj:
        list_origins = [k for k, v in destinations.items() if name in v]
        memory[name] = {k:0 for k in list_origins}

    return states, destinations, memory

states, destinations, memory = initialize(L)
print(states)
print(destinations)
print(memory)

{'a': 0, 'b': 0, 'c': 0}
{'broadcaster': ['a', 'b', 'c'], 'a': ['b'], 'b': ['c'], 'c': ['inv'], 'inv': ['a']}
{'inv': {'c': 0}}


In [117]:
# push button > low pulse to broadcaster
def advance(states, memory, pulses):
    #print(pulses)
    # pulses are (previous node, low/high, receiver)
    pulses = [(a, b, c) for a, b, c in pulses if c in destinations.keys()]
    for name in memory:
        for origin in memory[name]:
            for p in [p[1] for p in pulses if p[2] == name and p[0] == origin]:
                memory[name][origin] = p
    if len(pulses) == 0:
        return 0, 0, states
    new_pulses = []
    for name in set([p[2] for p in pulses]): 
        dest = destinations[name]  
        if name in memory.keys(): # conjunction module
            newp = 1 - all(memory[name].values())
            new_pulses.extend([(name, newp, d) for d in dest])
        elif name in states.keys(): #flipflop module
            p = [p[1] for p in pulses if p[2] == name][0]
            if p==0:
                states[name] = 1 - states[name]
                newp = states[name]
                new_pulses.extend([(name, newp, d) for d in dest])
        else:
            new_pulses.extend([(name, p[1], d) for d in dest for p in pulses if p[2] == name])

    k0, k1, res = advance(states, memory, new_pulses)
    k0 += len([p for p in new_pulses if p[1] == 0])
    k1 += len([p for p in new_pulses if p[1] == 1])
    return k0, k1, res

states, destinations, memory = initialize(data)

nk0, nk1 = 0, 0
for i in range(1000):
    nk0 += 1
    nk1 += 0 
    pulses = [('button', 0, 'broadcaster')]
    k0, k1, modules = advance(states, memory, pulses)
    nk0 += k0
    nk1 += k1
print(nk0, nk1, nk0*nk1)

17775 43921 780695775
